In [1]:
filename = 'N_11000_Y_648_X_486_1'
input_shape = (11000, 648, 486, 1)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers

import json
from os import path

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [4]:
sims_2d_path = '../data/2d_data/plane_sims/plane_sim_1100_648_486_1.npy'
planes_path = '../data/2d_data/planes/planes_1100_648_486_1.npy'
model_weights_path = '../models/2d_models/N_1100_Y_648_X_486/test1'

In [5]:
dataset_filename = '../data/2d_data/tf_records/plane_sims/plane_sims_N_10000_Y_648_X_486_1.tfrecords'

In [6]:
# Dataset path
record_dir = path.join('..', 'data', '2d_data', 'tf_records', 'plane_sims')
dataset_filename = path.join(record_dir, 'plane_sims_' + filename + '.tfrecords')

# Directory to store information about dataset
info_dir = path.join('..', 'data', '2d_data', 'tf_records', 'info')
info_sims_file = path.join(info_dir, filename + '_sims.json')
info_planes_file = path.join(info_dir, filename + '_planes.json')

# Layers

#### Shifted + scaled tanh activation function

In [7]:
# Batchnorm epsilon
BN_EPS = 1e-4

class ConvBnRelu2d(layers.Layer):
    def __init__(self, out_channels, kernel_size=(3, 3), padding='same'):
        super(ConvBnRelu2d, self).__init__()
        self.conv = layers.Conv2D(filters=out_channels, kernel_size=kernel_size, padding='same', use_bias=False)
        self.bn = layers.BatchNormalization(epsilon=BN_EPS)
        self.relu = layers.ReLU()
        
    def call(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

class TanhScale(layers.Layer):
    def __init__(self):
        super(TanhScale, self).__init__()
    
    def call(self, x):
        x = K.tanh(x)
        x = (x + 1) * 0.5
        return x * m

class StackEncoder(layers.Layer):
    def __init__(self, y_channels, kernel_size=3):
        super(StackEncoder, self).__init__()
        padding = (kernel_size - 1) // 2
        self.encode = keras.Sequential([
            ConvBnRelu2d(y_channels, kernel_size=kernel_size, padding='same'),
            ConvBnRelu2d(y_channels, kernel_size=kernel_size, padding='same')
        ])
        self.max_pool = layers.MaxPool2D(pool_size=2, strides=2)
        
    def call(self, x):
        x = self.encode(x)
        x_small = self.max_pool(x)
        return x, x_small


class StackDecoder(layers.Layer):
    def __init__(self, y_channels, kernel_size=3):
        super(StackDecoder, self).__init__()
        padding = (kernel_size - 1) // 2
        
        self.decode = keras.Sequential([
            ConvBnRelu2d(y_channels, kernel_size=kernel_size, padding='same'),
            ConvBnRelu2d(y_channels, kernel_size=kernel_size, padding='same'),
            ConvBnRelu2d(y_channels, kernel_size=kernel_size, padding='same'),
            ]
        )
        
        
    def call(self, x, down_tensor):
        x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
        
        # Calculate cropping for down_tensor to concatenate with x
        _, h2, w2, _ = down_tensor.shape
        _, h1, w1, _ = x.shape
        h_diff, w_diff = h2 - h1, w2 - w1
        
        cropping = ((int(np.ceil(h_diff / 2)), int(np.floor(h_diff / 2))),
                    (int(np.ceil(w_diff / 2)), int(np.floor(w_diff / 2))))
        down_tensor = layers.Cropping2D(cropping=cropping)(down_tensor)        
        x = layers.concatenate([x, down_tensor], axis=3)
        x = self.decode(x)
        return x

# Model

In [8]:
class UNet_2d(keras.Model):
    def __init__(self):
        super(UNet_2d, self).__init__()

        self.down1 = StackEncoder(24, kernel_size=3) # 256
        self.down2 = StackEncoder(64, kernel_size=3)  # 128
        self.down3 = StackEncoder(128, kernel_size=3)  # 64
        self.down4 = StackEncoder(256, kernel_size=3)  # 32
        self.down5 = StackEncoder(512, kernel_size=3)  # 16
        

        self.up5 = StackDecoder(256, kernel_size=3)  # 32
        self.up4 = StackDecoder(128, kernel_size=3)  # 64
        self.up3 = StackDecoder(64, kernel_size=3)  # 128
        self.up2 = StackDecoder(24, kernel_size=3)  # 256
        self.up1 = StackDecoder(24, kernel_size=3)  # 512
        self.classify = layers.Conv2D(filters=1, kernel_size=1, use_bias=True)
        
        self.center = ConvBnRelu2d(512, kernel_size=3, padding='same')
        
        
    def call(self, x):
        out = x;
        down1_tensor, out = self.down1(out)
        down2_tensor, out = self.down2(out)
        down3_tensor, out = self.down3(out)
        down4_tensor, out = self.down4(out)
        down5_tensor, out = self.down5(out)

        out = self.center(out)
        
        out = self.up5(out, down5_tensor)
        out = self.up4(out, down4_tensor)
        out = self.up3(out, down3_tensor)
        out = self.up2(out, down2_tensor)
        out = self.up1(out, down1_tensor)

        out = self.classify(out)
        out = tf.squeeze(out, axis=3)
        
        return out

In [9]:
#         out = self.activation_fn(out)
#         self.activation_fn = TanhScale()
#       self.bn = layers.BatchNormalization(epsilon=BN_EPS)

# Data Loading, Preprocessing

In [ ]:
def normalize_N_11000(x, eps):
    return (x - MEANS) / (STDS + eps)

In [19]:
obj_dims = (648, 486)

In [ ]:
### Preprocessing for N=11000, using npz files
sims_2d_path = '../data/2d_data/plane_sims/plane_sim_11000_648_486_1.npz'
planes_uint_path = '../data/2d_data/planes/planes_uint_11000_648_486_1.npz'

In [ ]:
sims_arrays = np.load(sims_2d_path)

In [ ]:
MEANS = sims_arrays['means']
STDS = sims_arrays['stds']

In [ ]:
# Load in forward model simulations
sims_2d = sims_arrays['plane_sims']
n, y, x = sims_2d.shape
sims_2d = sims_2d[..., np.newaxis] # Expand to channel dimension
sims_2d = sims_2d.astype(np.float16) # Cast to float16
input_shape = sims_2d.shape

# Load in ground truth
planes = np.load(planes_path)['planes']

# Crop to target image size
m = 1
planes = planes[:, 4:644, 3:483]

# Standard normalize input/ground truth
eps = 1e-4

# planes = normalize_N_11000(planes, eps)
sims_2d = normalize_N_11000(sims_2d, eps)

# TFRecords Approach

In [13]:
with open(info_planes_file) as planes_f:
    data = json.load(planes_f)
    PLANE_MEANS = np.array(data['means'], dtype=np.float16)
    PLANE_STDS = np.array(data['stds'], dtype=np.float16)

with open(info_sims_file) as sims_f:
    data = json.load(sims_f)
    SIM_MEANS = np.array(data['means'], dtype=np.float16)
    SIM_STDS = np.array(data['stds'], dtype=np.float16)

[[0.000e+00 9.090e-05 2.728e-04 ... 3.636e-04 3.636e-04 9.090e-05]
 [9.090e-05 1.818e-04 2.728e-04 ... 7.272e-04 7.272e-04 1.818e-04]
 [4.547e-04 5.455e-04 1.818e-04 ... 4.547e-04 4.547e-04 1.818e-04]
 ...
 [2.728e-04 5.455e-04 6.366e-04 ... 8.183e-04 1.091e-03 5.455e-04]
 [2.728e-04 4.547e-04 4.547e-04 ... 1.182e-03 9.093e-04 2.728e-04]
 [9.090e-05 1.818e-04 3.636e-04 ... 5.455e-04 1.818e-04 0.000e+00]]


In [82]:
def _parse_function(example_proto):
    eps = 1e-1
    feature_description = {
        'plane': tf.io.FixedLenFeature(obj_dims, tf.float32),
        'sim': tf.io.FixedLenFeature(obj_dims, tf.float32)
    }
    # Parse the input tf.Example proto using the dictionary above.
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    plane = example['plane']
    plane = tf.cast(plane, tf.float16)
    plane = (plane - PLANE_MEANS) / (PLANE_STDS + eps) # Normalizzation
    plane = plane[4:644, 3:483] # Crop to target image size

    sim = example['sim']
    sim = tf.cast(sim, tf.float16)
    sim = (sim - SIM_MEANS) / (SIM_STDS + eps) # Normalization
    sim = sim[..., np.newaxis] # Expand to channel dimension
    
    return sim, plane

In [83]:
def create_dataset(filenames):
    raw_dataset = tf.data.TFRecordDataset(filenames=dataset_filename)
    
    dataset = raw_dataset.map(_parse_function)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(256)
    dataset = dataset.batch(BATCH_SIZE)
    
    return dataset

# Old code

In [61]:
def normalize(x, eps):
    return (x - np.mean(x, axis=0)) / (np.std(x, axis=0) + eps)

In [ ]:
# Load in forward model simulations
sims_2d = np.load(sims_2d_path)
n, y, x = sims_2d.shape
sims_2d = sims_2d[..., np.newaxis] # Expand to channel dimension
sims_2d = sims_2d.astype(np.float32) # Cast to float32
input_shape = sims_2d.shape

# Load in ground truth
planes = np.load(planes_path)

# Crop to target image size
m = 1
planes = planes[:, 4:644, 3:483]

# Standard normalize input/ground truth
eps = 1e-4

planes = normalize(planes, eps)
sims_2d = normalize(sims_2d, eps)

In [ ]:
planes = normalize(planes, eps)
sims_2d = normalize(sims_2d, eps)

In [ ]:
# assert planes.min() == 0.0 and planes.max() == m, "planes min/max scaling issue"
# assert sims_2d.min() == 0.0 and sims_2d.max() == m, "sims_2d min/max scaling issue"

In [ ]:
plt.title("Sample input")
plt.imshow(sims_2d[0,:,:,0])
plt.show()

# Logging

In [49]:
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

def scaled_mse_loss(y_actual,y_pred):
    loss = K.square((y_actual-y_pred))
#     loss = K.sqrt(loss)
    loss = K.sum(loss)
    return loss

def plot_image_tensorboard(epoch, logs):
    # Create a plot to visualize image reconstruction progress
    
    # Call the model to get prediction
    dataset_iter = iter(train_dataset)
    example = next(dataset_iter)
    pred = model.predict(example['sim'][np.newaxis])

    # Create a mpl figure
    figure = plt.figure(figsize=(10,10))

    # Plot the prediction
    plt.subplot(1, 2, 1)
    plt.title("prediction")
    plt.imshow(pred[0])
    # Plot groundtruth
    plt.subplot(1, 2, 2)
    plt.title("ground truth")
    plt.imshow(example['plane'])
    plot_image = plot_to_image(figure)
    with file_writer.as_default():
        tf.summary.image("Prediction vs Ground Truth", plot_image, step=epoch)

import datetime
log_dir=os.path.join('logs_2d', 'fit') + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=1000000)
file_writer = tf.summary.create_file_writer(log_dir)

plot_image_tensorboard_cb = keras.callbacks.LambdaCallback(on_epoch_end=plot_image_tensorboard)

# Training

In [56]:
model = UNet_2d()
adam = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, amsgrad=False)
# model.compile(optimizer=adam, loss=scaled_mse_loss, metrics=['mean_squared_error'])
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
model.build(input_shape)
model.summary()

Model: "u_net_2d_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
stack_encoder_10 (StackEncod multiple                  5592      
_________________________________________________________________
stack_encoder_11 (StackEncod multiple                  51200     
_________________________________________________________________
stack_encoder_12 (StackEncod multiple                  222208    
_________________________________________________________________
stack_encoder_13 (StackEncod multiple                  886784    
_________________________________________________________________
stack_encoder_14 (StackEncod multiple                  3543040   
_________________________________________________________________
stack_decoder_10 (StackDecod multiple                  3542016   
_________________________________________________________________
stack_decoder_11 (StackDecod multiple                  8

In [ ]:
EPOCHS = 200
BATCH_SIZE = 8
TRAIN_LENGTH = 10000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

train_x = sims_2d[:TRAIN_LENGTH]
train_y = planes[:TRAIN_LENGTH]

model.fit(x=train_x, y=train_y, epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, callbacks=[plot_image_tensorboard_cb, 
                                                                                           tensorboard_callback])

# TFRecordDataset Training

In [85]:
TRAIN_LENGTH = 20
VAL_LENGTH = 8
TEST_LENGTH = 0
EPOCHS = 200
BATCH_SIZE = 4
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

dataset = create_dataset(filenames=dataset_filename)
train_dataset = dataset.take(TRAIN_LENGTH)

val_dataset = dataset.skip(TRAIN_LENGTH)
val_dataset = dataset.take(VAL_LENGTH)

test_dataset = dataset.skip(VAL_LENGTH + TEST_LENGTH)
test_dataset = dataset.take(TEST_LENGTH)


In [86]:
print(next(iter(train_dataset)))

(<tf.Tensor: shape=(4, 648, 486, 1), dtype=float16, numpy=
array([[[[185.   ],
         [143.8  ],
         [152.2  ],
         ...,
         [163.9  ],
         [195.9  ],
         [282.8  ]],

        [[288.5  ],
         [284.2  ],
         [185.5  ],
         ...,
         [235.   ],
         [179.2  ],
         [117.1  ]],

        [[224.1  ],
         [366.2  ],
         [358.   ],
         ...,
         [298.8  ],
         [281.   ],
         [255.9  ]],

        ...,

        [[817.5  ],
         [720.5  ],
         [408.5  ],
         ...,
         [669.   ],
         [617.   ],
         [824.5  ]],

        [[276.2  ],
         [343.2  ],
         [341.   ],
         ...,
         [363.5  ],
         [270.5  ],
         [278.   ]],

        [[173.5  ],
         [130.6  ],
         [257.8  ],
         ...,
         [307.2  ],
         [360.5  ],
         [194.2  ]]],


       [[[-54.62 ],
         [-65.9  ],
         [ 67.56 ],
         ...,
         [252.2  ],
         [ 19.4

In [87]:



model.fit(train_dataset, epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, 
          callbacks=[plot_image_tensorboard_cb, tensorboard_callback], validation_data=val_dataset, validation_steps=5)

Train for 5 steps, validate for 5 steps
Epoch 1/200
2/5 [===========>..................] - ETA: 21s - loss: 0.9351 - mean_squared_error: 0.9351

TypeError: tuple indices must be integers or slices, not str

In [ ]:
model.save_weights(model_weights_path, save_format='tf')

# Timing Tests 

In [ ]:
model = UNet_2d()
model.load_weights(model_weights_path)

In [ ]:
import time

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
j = 995
num_predict = 10

t0 = time.perf_counter()
pred = model.predict(sims_2d[j:j+num_predict])
t1 = time.perf_counter()

print('Prediction time per sample:', (t1 - t0) / num_predict, 's')

In [ ]:
fig=plt.figure(figsize=(30, 15 * num_predict))
rows, columns = num_predict, 2
for i in range(num_predict):
    # Plot prediction
    s = fig.add_subplot(rows, columns, columns*i+1)
    s.set_title("prediction {0}".format(j + i), size=20)
    plt.imshow(pred[i])
    
    # Plot ground truth
    s = fig.add_subplot(rows, columns, columns*i+2)
    s.set_title("ground truth {0}".format(j + i), size=20)
    plt.imshow(planes[j + i])
    
plt.savefig('N_1000_epochs_200.pdf')